In [ ]:
pip -q install peft transformers langchain sentencepiece

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from langchain.prompts import PromptTemplate

In [ ]:
adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
device_map = {"": 0}

model = AutoPeftModelForCausalLM.from_pretrained(adapt_model_name, device_map=device_map, torch_dtype=torch.bfloat16)

In [ ]:
# Load model from HuggingFace Hub
sent_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
sent_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def get_embedding(sentence):

    #Mean Pooling - Take attention mask into account for correct averaging
    def _mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Tokenize sentences
    encoded_input = sent_tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = sent_model(**encoded_input)

    # Perform pooling
    sentence_embeddings = _mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings

In [ ]:
info_prompt_less10 = PromptTemplate.from_template("user: Небольшой свечной завод, расположенный по адресу ул. Ленина 22, находится в городе Градск. Доехать до завода от центрального автовокзала можно на автобусе номер 666. Завод специализируется на производстве свечей различных размеров и форм. Открытая вакансия: Производственный работник - зарплата 150 тысяч рублей в месяц. Требования: 6 разряд свечника, 25 лет опыта. Претендент на работу должен заполнить анкету, предоставить резюме, пройти собеседование с представителями компании. Для работников завода предусмотрены следующие бонусы: Медицинское страхование, Абонемент в бассейн. Завод активно проводит праздничные мероприятия и игры. {question}\nbot: Вот ответ на ваш вопрос длиной не более 10 слов:")

In [ ]:
def get_answer(info_prompt, question):

    prompt = info_prompt.format(question=question)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                            top_p=0.5,
                            temperature=0.3,
                            attention_mask=inputs["attention_mask"],
                            max_new_tokens=100,
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True)

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    parsed_answer = output.split("Вот ответ на ваш вопрос длиной не более 10 слов:")[1].strip()

    if "bot:" in parsed_answer:
        parsed_answer = parsed_answer.split("bot:")[0].strip()

    # parsed_answer = output.split("bot:")[1].strip()
    return parsed_answer

In [ ]:
def get_cos_sim(question):
    cos_sim = F.cosine_similarity(emb_database, emb, dim=1, eps=1e-8)
    return cos_sim

In [ ]:
answers = []
emb_database = torch.empty((0, 384), dtype=torch.float32)

In [ ]:
question = "Каких работников вы ищете?"
emb = get_embedding(question)

In [ ]:
get_cos_sim(question)

tensor([])

In [ ]:
emb = get_embedding(question)
cos_sim = get_cos_sim(question)
answer = get_answer(info_prompt_less10, question)
emb_database = torch.cat((emb_database, emb), 0)
answers.append(answer)
print(f'Answer from model: {answer}')

Answer from model: Производственный работник.


In [ ]:
question = "Где находится ваш завод?"
emb = get_embedding(question)

In [ ]:
get_cos_sim(question)

tensor([0.5328])

In [ ]:
emb = get_embedding(question)
cos_sim = get_cos_sim(question)
answer = get_answer(info_prompt_less10, question)
emb_database = torch.cat((emb_database, emb), 0)
answers.append(answer)
print(f'Answer from model: {answer}')

Answer from model: Завод находится по адресу ул. Ленина 22, Градск.


In [ ]:
question = "Где расположен ваш завод?"
emb = get_embedding(question)
cos_sim = get_cos_sim(question)
cos_sim

tensor([0.5483, 0.8426])

In [ ]:
print(f'{cos_sim} {answers=}')

tensor([0.5483, 0.8426]) answers=['Производственный работник.', 'Завод находится по адресу ул. Ленина 22, Градск.']


In [ ]:
questions = [
"Какой адрес вашего завода?",
"Где находится ваш завод?",
"Какое местоположение вашего завода?",
"Где точно находится ваш завод?",
"Как добраться до вашего завода?",
"Как мне добраться до вашего завода?",
"На каком автобусе добраться до вашего завода?",
"Какие автобусы едут до вашего завода?",
"Что производят на вашем заводе?",
"Какая продукция производится на вашем заводе?",
"Какие товары производятся на вашем заводе?",
"Что именно производится на вашем заводе?",
"Какие изделия производятся на вашем заводе?",
"Какую продукцию я могу найти на вашем заводе?",
"Какик работники вам нужны?",
"Какие должности вы ищете?",
"Какие работники вам нужны в данный момент?",
"Какие специалисты вам требуются?",
"Какие специалисты вам необходимы?",
"Какими навыками и умениями должен обладать специалист?",
"Какие компетенции и знания необходимы специалисту?",
"Какими навыками и качествами должен владеть специалист?",
"Какие специфические умения и опыт требуются у специалиста?",
"Сколько платят на этой должности?",
"Какова зарплата для этой должности?",
"Какая заработная плата предлагается?",
"Какие условия оплаты?",
"Какие варианты и размеры заработной платы доступны для этой должности?",
"Какая зарплата ожидается для новых сотрудников на этой позиции?",
"Как можно устроиться к вам на работу?",
"Как устроиться к вам на работу?"
"Какой процесс приема на работу?",
"Как я могу подать заявку на работу на вашем заводе?",
"Какие шаги мне нужно предпринять, чтобы устроиться к вам на завод?",
"Что сделать, чтобы устроиться к вам на завод?",
"Какие дополнительные бонусы и привилегии предоставляются?",
"Какие бонусы и вознаграждения доступны для сотрудников?",
"Какие бонусные программы предоставляются?",
"Какие дополнительные бонусы и предлагаются?"
]

In [ ]:
# Find the index of the maximum value
for q in questions:
    print(q)
    emb = get_embedding(q)
    cos_sim = get_cos_sim(q)
    max_value, max_index = torch.max(get_cos_sim(q), dim=0)

    if max_value > 0.83:
        answer = answers[max_index]
        print(f'DATABASE: {answer}')
    else:
        answer = get_answer(info_prompt_less10, q)
        emb_database = torch.cat((emb_database, emb), 0)
        answers.append(answer)
        print(f'MODEL: {answer}')
    print()

Какой адрес вашего завода?
MODEL: ул. Ленина 22, Градск.

Где находится ваш завод?
DATABASE: Завод находится по адресу ул. Ленина 22, Градск.

Какое местоположение вашего завода?
MODEL: ул. Ленина 22, Градск.

Где точно находится ваш завод?
DATABASE: Завод находится по адресу ул. Ленина 22, Градск.

Как добраться до вашего завода?
MODEL: "Автобус номер 666".

Как мне добраться до вашего завода?
DATABASE: "Автобус номер 666".

На каком автобусе добраться до вашего завода?
DATABASE: "Автобус номер 666".

Какие автобусы едут до вашего завода?
DATABASE: ул. Ленина 22, Градск.

Что производят на вашем заводе?
MODEL: Свечи различных размеров и форм.

Какая продукция производится на вашем заводе?
DATABASE: Свечи различных размеров и форм.

Какие товары производятся на вашем заводе?
DATABASE: Свечи различных размеров и форм.

Что именно производится на вашем заводе?
DATABASE: Свечи различных размеров и форм.

Какие изделия производятся на вашем заводе?
DATABASE: Свечи различных размеров и форм